<a href="https://colab.research.google.com/github/Drisnor/IA/blob/master/M2_RFA_TP1_etudiant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# M2 IARF-RODECO TP 1 : classification et régression avec un MLP/FCNN

MLP : Multi-Layer Perception ou Perceptron Multi-Couche en français

Un MLP peut s'appeler également FCNN pour Fully-Connected Neural Network




**Auteurs :** Benjamin Chamand - Thomas Pellegrini

**Contributeur :** Lionel Pibre (sept 2020)

**Année de création :** 2018

## Gestion des pré-requis avant de commencer le TP

### Importation des bibliothèques

Premièrement, il faut faire les import nécessaires au TP :

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import np_utils
from keras import backend as K
from keras import activations

from distutils.version import LooseVersion as LV
from keras import __version__

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import pandas as pd

print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))

Si on utilise le backend Tensorflow (ce qui est le cas par défaut) on peut facilement avoir des informations sur le CPU ou le GPU accessible.

Dans un premier temps, vérifier bien que keras utilise le GPU pour les calculs.
Si ce n'est pas le cas, il faut aller dans Exécution > Modifier le type d'exécution > Accélérateur Matériel > GPU

In [ ]:
if K.backend() == "tensorflow":
    from tensorflow.python.client import device_lib
    print(device_lib.list_local_devices())

### Fixer la seed

Afin de reproduire nos résultats obtenus à chaque fois et ainsi voir si nos modifications apportent une réelle amélioration au système de base, il faut fixer la *seed* !
La *seed* est un nombre qui est utilisé pour l'initialisation du générateur de nombres pseudo-aléatoires.

Sur Keras avec Tensorflow en backend, il faut fixer la *seed* des bibliothèques *Numpy* et *Tensorflow*.

In [ ]:
from numpy.random import seed
seed(123) # On peut utiliser n'importe quel nombre

import tensorflow
tensorflow.random.set_seed(1234)

## Partie I : Classification avec le dataset Fashion-MNIST

Durant la première partie, nous allons mettre en place un réseau de neurones de type MLP (Multi-Layer Perceptron) afin de résoudre une tâche de classication.

Cette tâche consiste de classifier correctement des images de vêtements (pull, pantalon, jupe, sac, ...) issues du corpus Fashion-MNIST.

#### Téléchargement et affichage des données

**Téléchargement du corpus**

La première étape consiste à télécharger le dataset *Fashion-MNIST*.

Le corpus *Fashion-MNIST* a été réalisé par l'équipe de recherche de Zalando proposant une alternative au corpus *MNIST* en utilisant les mêmes tailles des images et la même structure. Il est constitué d'articles vendu par Zalando.

Il est donc facile de tester notre réseau entre *MNIST* et *Fashion-MNIST* en changeant 2 lignes de code.

In [ ]:
# Chargement du dataset MNIST:
#from keras.datasets import mnist
#(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Chargement du dataset Fashion-MNIST:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

**Taille du corpus**

Affichage de la taille du corpus

In [ ]:
# Affichage du nombre d'exemples totales dans le corpus
print('Taille du corpus total')
print('\t• train :', len(x_train), 'exemples')
print('\t• test :', len(x_test), 'exemples')

# Affichage de la taille des images et des labels dans le corpus 
print('\nTaille des données d\'apprentissage')
print('\t• X_train (images) :', x_train.shape)
print('\t• y_train (labels) :', y_train.shape)

print('\nTaille des données de test')
print('\t• X_test (images) :', x_test.shape)
print('\t• y_test (labels) :', y_test.shape)

**Correspondance ID du label / signification**

Définition de la liste *idx_to_classes* permettant à partir de la valeur du label de retrouver sa signification

In [ ]:
# liste de conversion dans le cas de MNIST
#idx_to_classes = ['zero', 'un', 'deux', 'trois', 'quatre',
#                  'cinq', 'six', 'sept', 'huit', 'neuf']

# liste de conversion dans le cas de Fashion-MNIST
idx_to_classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                  'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Anlke boot']

"""
Affichage des labels des 10 premières images d'entraînement
"""
# exemple en affichant la correspondance des labels sur 10 premières données
# du corpus d'apprentissage
print("Affichage de la correspondance des labels :")
for i in range(10):
    print('• y_train[' + str(i) + '] =', y_train[i], '->', idx_to_classes[y_train[i]])

**Affichage des images**

On affiche maintenant quelques images issues du corpus

In [ ]:
plt.figure(figsize=(12,4))
for i in range(14):
    # récupération d'une image et de son label associé
    img, target = x_train[i], y_train[i]
    # choix de la zone d'affichage et afficage de l'image en niveau de gris
    plt.subplot(2,7,i+1)
    plt.imshow(img, cmap="gray")
    # ajout d'un titre à l'image    
    plt.title('{} ({})'.format(idx_to_classes[target], target))
    plt.axis('off')
              
plt.show()

#### Prétraitements des données

Dans cette section, nous allons voir quelles sont les prétraitements basique à réaliser sur les données avant de les alimenter à notre réseau de neurones.

**Conversion en float**

Afin de faciliter les différentes opérations de traitements sur les images, on va convertir les images d'apprentissage et de test en type float 32 bits.

*Question* : quel est le type des images x_train et x_test pour l'instant ? 
Pour répondre, afficher le tableau de la première image dans x_train. Quel est l'intervalle de valeurs possible des pixels ?

In [ ]:
# afficher les valeurs des pixels de la première image de x_train
img, target = x_train[0], y_train[0]
for i in range(10):
    print(img[i])  # ndarray d'entiers [0, 254]


In [ ]:
# Conversion des données en type float32:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
"""
img, target = x_train[0], y_train[0]
for i in range(10):
    print(img[i])
"""

**Encodage one-hot**

Comme il a été présenté dans le cours, lors d'une tâche de classification, la sortie du réseau dispose de $n$ neurones correspondants au $n$ classes que l'on désire en sortie.

Afin de calculer notre fonction de coût pour ensuite appliquer l'algorithme de descente de gradient sur notre réseau, il faut au préalable encoder les labels dans le format **one-hot** qui consiste à représenter notre label en vecteur de valeur binaire dont sa représentation n'a qu'un seul 1.

In [ ]:
nb_classes = 10

# one-hot encoding
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# affichage de l'encodage des labels sur les 10 premières données
# du corpus d'apprentissage
print("Affichage de l'encodage des labels :")
for i in range(10):
    print('• y_train[' + str(i) + '] =', y_train[i], '->', Y_train[i])

**Normalisation des données**

Dans le domaine du traitement des images, de nombreuses méthodes peuvent être utilisées, en voici deux exemples :

* **rééchelonnage :** normalisation linéaire consistant à étaler la dynamique totale de l'image dans un nouvel intervalle. Dans notre cas, l'intervalle est \[0,1\].
* **standardisation :** normalisation linéraire consistant à soustraire la moyenne et diviser par l'écart-type.



*Exercice* définir la fonction rééchelonnage ou la standardisation (ou les 2), puis l'appliquer sur vos images d'apprentissage et de tests

In [ ]:
# définition du rééchelonnage
def reechelonnage(X_train, X_test):
    for i in X_train :
      i /= 255
    for j in X_test :
      j /= 255
    return X_train, X_test
"""
x_train, x_test = reechelonnage(x_train, x_test)
print(x_train[5], "\n\n")
print(x_test[5])
"""

In [ ]:
# définition de la standardisation
def standardisation(X_train, X_test):
    m = np.mean(X_train)
    e = np.nanstd(X_train)
    X_train = (X_train - m) / e

    X_test = (X_test - m) / e
    return X_train, X_test

x_train, x_test = standardisation(x_train, x_test)
#print(x_train[5], "\n\n")
#print(x_test[5])

On applique une des normalisations définies précédemment sur nos images du corpus d'apprentissage et de tests

In [ ]:
# sans normalisation
#X_train, X_test = x_train, x_test

# normalisation des données
X_train, X_test = standardisation(x_train, x_test)

**Applatissement des images**

Comme vous l'avez vu précédemment lors du calcul de l'affichage des dimensions des données de notre corpus, les images sont représentées par une matrice 2D.
Dans le cas d'un réseau de neurones MLP/FCNN, l'entrée du réseau est un vecteur de neurones et non une matrice 2D de neurones, il faut donc vectoriser nos données en entrée de notre réseau.

Pour cela, on pourra s'aider de la fonction *reshape* afin de modifier la taille des données (pour "applatir" la matrice 2D en vecteur 1D).

In [ ]:
def img_reshape(X_train, X_test):
    X_train = X_train.reshape((-1,28*28))
    X_test = X_test.reshape((-1,28*28))
    return X_train, X_test

In [ ]:
X_train, X_test = img_reshape(X_train, X_test)

print('\nTaille des données :')
print('\t• X_train :', X_train.shape)
print('\t• X_test :', X_test.shape)

#### Création et apprentissage d'un modèle MLP

Dans cette section, nous allons nous intéresser à la définition d'un modèle MLP puis lancer l'apprentissage en utilisant la bibliothèque Keras.

**Définition du modèle**

Dans la cellule suivante, on va définir un modèle qui va être entraîné sur le corpus Fashion-MNIST afin de classifier suivant les 10 classes que l'on souhaite.

Tout d'abord, lire cette page de documentation de Keras qui vous explique comment définir un modèle "Sequential" : 

https://keras.io/getting-started/sequential-model-guide/

In [ ]:
# Model initialization:
model = Sequential()

# Notre premier modele avec une seule couche cachée de 50 neurones.

# Pour le construire, ajouter une couche Dense. 
# Attention vous devez préciser la dimension des inputs pour cette première couche :
#model.add(keras.Input(shape=(784,)))
model = keras.Sequential(
    [
        Dense(50, activation="relu", name="layer1", input_shape=(784,))
    ]
)

# Ensuite ajouter une activation de type ReLu :
#model.add(Activation(activations.relu))

# Ajouter maintenant la couche de sortie avec le bon nombre de neurones et la bonne fonction d'activation :
model.add(Dense(10, activation="softmax", name="output"))

# Compléter la ligne suivante en choisissant la bonne fonction de coût.
# Voir la liste des fonctions de coût disponibles ici :
# https://keras.io/losses/

model.compile(loss='categorical_crossentropy',  #binary_crossentropy avec Keras
              optimizer='rmsprop', 
              metrics=['accuracy'])

In [ ]:
# Affichage des informations sur le réseau défini au-dessus
model.summary()

**Mode apprentissage**

Lancer l'apprentissage du modèle sur nos données. Compléter les arguments de la méthode *fit* ci-dessous

In [ ]:
%%time
epochs = 10 

# compléter les arguments nécessaires ci-dessous
history = model.fit(X_train, Y_train, epochs=10)

**Affichage de données issus de l'apprentissage**

On va maintenant afficher graphiquement des informations que l'on a récupéré après l'apprentissage de notre modèle


In [ ]:
plt.figure(figsize=(11,3))

# affichage de la valeur de la fonction de perte
plt.subplot(1,2,1)
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

# affichage de la précision de notre réseau sur les données d'apprentissage
plt.subplot(1,2,2)
plt.plot(history.epoch,history.history['accuracy'])
plt.title('accuracy');

Que pouvez-vous dire sur ces courbes ?

### Inférence des données de tests

On va maintenant évaluer notre modèle sur des exemples non vus pendant l'apprentissage.
Pour cela, il suffit juste de présenter nos données au modèle sans réaliser la descente de gradient, on appelle cela faire une inférence, c'est-à-dire que l'on va juste faire des prédictions sur nos données.

In [ ]:
%%time
scores = model.evaluate(X_test, Y_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Ce score est-il bon ? 

**Affichage des erreurs de prédictions**

Regardons des erreurs de prédictions faites par le modèle

In [ ]:
def show_failures(predictions, trueclass=None, predictedclass=None, maxtoshow=10):
    rounded = np.argmax(predictions, axis=1)
    errors = rounded!=y_test
    print('La prédiction est affichée en premier et entre parenthèses, le vrai label.')
    ii = 0
    plt.figure(figsize=(maxtoshow+maxtoshow/4, 1))
    for i in range(x_test.shape[0]):
        if ii>=maxtoshow:
            break
        if errors[i]:
            if trueclass is not None and y_test[i] != trueclass:
                continue
            if predictedclass is not None and rounded[i] != predictedclass:
                continue
            plt.subplot(1, maxtoshow, ii+1)
            plt.axis('off')
            img = X_test[i].reshape(28, 28)
            plt.imshow(img, cmap="gray")
            plt.title("{}\n({})".format(idx_to_classes[rounded[i]], idx_to_classes[y_test[i]]))
            ii = ii + 1

In [ ]:
predictions = model.predict(X_test)

show_failures(predictions)

In [ ]:
# pour afficher les erreurs du chiffre 6 uniquement :
show_failures(predictions, trueclass=6)

En général, dans une tâche de classification en apprentissage supervisé, on affiche la matrice de confusions pour présenter les résultats.

In [ ]:
from sklearn.metrics import confusion_matrix

print('Confusion matrix (rows: true classes; columns: predicted classes):'); print()
cm=confusion_matrix(y_test, np.argmax(predictions, axis=1), labels=list(range(10)))
print(cm); print()

print('Classification accuracy for each class:'); print()
acc = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
for i,j in enumerate(acc.diagonal()):
    print("%d: %.4f" % (i,j))

On la lit ** par lignes** !

In [ ]:
# afficage de la matrice de confusions dans un autre format
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.hot_r)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(idx_to_classes))
plt.grid(None)
classes_name = zip(idx_to_classes, list(range(len(idx_to_classes))))
classes_name = ['{} ({})'.format(*i) for i in classes_name]
plt.xticks(tick_marks, classes_name, rotation=45)
plt.yticks(tick_marks, classes_name)

thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

 **Playground**

Vous allez essayer d'améliorer la performance de votre modèle simple.

Reprendre et modifier votre MLP pour améliorer le taux de bonne 
classification ou pour voir l'influence des paramètres : 

*  en ajoutant une ou plusieurs couches cachées,

* en augmentant le nombre de neurones,

* en changeant la fonction d'activation des couches cachées,

* en rendant plus robuste votre MLP avec des couches de "dropout" et en testant différents taux de dropout

* Consulter la documentation de Keras https://keras.io/, par exemple sur les couches Dense, Activation et Dropout : https://keras.io/layers/core/

* Vous pouvez aussi jouer sur le nombre d'epochs d'apprentissage, la taille du mini-batch, changer l'optimizer, etc. Voir https://keras.io/optimizers/



### (Optionnel) Codage de la fonction de coût (_loss function_)
Lors de l'utilisation de la méthode **_compile_** de votre modèle précèdent, vous avez spécifié une fonction de perte pour le calcul des gradients qui était la __*categorical\_crossentropy*__ définis par la fonction suivante :

$$
- \sum_{i=0}^{N} y_i \log(\hat{y}_i)
$$
*Pas diviser par 1/N !!*

avec:
* $N$ : le nombre totale de classes
* $y$ : la sortie du réseau voulue
* $\hat{y}$: la sortie du réseau estimée

**Exercice :** dans la cellule suivante, essayer de recoder la fonction de coût *categorical_crossentropy* et l'utiliser dans votre modèle précédent pour voir si elle fonctionne

Aide :
* Toutes les fonctions à utiliser sont dans le backend de Keras. Exemple : pour faire une somme, utiliser K.sum()
* Attention, il faut limiter les valeurs du log dans [1e-7, 1.0-1e-7] par exemple. Pour cela utiliser K.clip()
* Les dimensions de la vérité terrain et des prédictions est (batch_size, nb_classes). La fonction s'applique sur chaque élément du batch séparément et renvoie un tenseur de dimension (batch_size, ).


In [ ]:
def categorical_crossentropy(y_true, y_pred):
    # Permet d'éviter les problèmes avec le log quand y_pred tend vers 0 
    # A faire : limiter les valeurs de y_pred :
    y_pred = K.clip(y_pred, 1e-7, 1.0-1e-7)
    batch_size, nb_classes = y_pred.shape[0], y_pred.shape[1]
    # Calcul de l'entropie croisée
    cost = [0]*batch_size
    for i in range(batch_size):
      cost[i] = - K.sum(y_true[i] * K.log(y_pred[i]))  #1/nb_classes *
    return cost

**Test :** comparaison avec la fonction de coût définie dans Keras

In [ ]:
y_true = K.variable(value=np.array([[1, 0, 0],
                                    [0, 0, 1]]))

y_pred = K.variable(value=np.array([[0, 0.5, 0.5],
                                    [0.8, 0.1, 0.1]]))

# remarque : toutes les opérations faisant appel au backend K sont symboliques. 
# Il faut donc utiliser K.eval() pour exécuter les opérations sur des tenseurs numériques

loss1 = K.eval(K.categorical_crossentropy(y_true, y_pred, from_logits=False))
loss2 = K.eval(categorical_crossentropy(y_true, y_pred))
print(loss1)
print(loss2)

# vous devez obtenir True en sortie de ce test 
print("Test de validité : " + str(K.eval(K.all(loss1 == loss2))))

Remplacer à présent la valeur de l'option _loss_ de la méthode _compile_ avec votre fonction de coût.
Vérifier que l'apprentissage se passe correctement.

## Partie II : Régression sur le dataset Boston Housing Prices

Pour cette deuxième partie, nous allons nous intéresser à une autre tâche de l'apprentissage machine qui est la régression de données, c'est-à-dire que l'on va estimer la valeur d'une variable par rapport à une ou plusieurs autres variables.

Pour résoudre cette tâche, nous allons utiliser le corpus *Boston Housing Prices* qui consiste à partir de 13 attributs de maisons situées à différents endroits dans la banlieue de Boston à la fin des années 1970, de trouver les valeurs médianes du prix des maisons à un emplacement de Boston (en k$).

#### Téléchargement et affichage des données

**Téléchargement du corpus**

Comme dans la première partie, on télécharge le dataset *Boston Housing Prices* dont l'interface de chargement des données est présente dans la bibliothèque Keras.

Ce corpus a été réalisé par Harrison and Rubinfeld dans les années 70.

In [ ]:
from keras.datasets import boston_housing

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

**Taille du corpus**

Affichage de la taille du corpus

In [ ]:
# Affichage du nombre d'exemples totales dans le corpus
print('Taille du corpus total')
print('\t• train :', len(x_train), 'exemples')
print('\t• test :', len(x_test), 'exemples')

# Affichage de la taille des attributs du corpus 
print('\nTaille des données d\'apprentissage')
print('\t• X_train (attributs) :', x_train.shape)
print('\t• y_train (valeur à estimer) :', y_train.shape)

print('\nTaille des données de test')
print('\t• X_test (attributs) :', x_test.shape)
print('\t• y_test (valeur à estimer) :', y_test.shape)

**Informations sur le corpus**

Comme vous avez pu voir lors de la visualisation dans la cellule précédente, chaque élément du corpus dispose de 13 attributs qui sont les suivants :

- **CRIM :** taux de criminalité par habitant par ville ;
- **ZN :** proportion de terrains résidentiels zonés pour les lots de plus de 25 000 pi.ca. (pied carré, système de mesure américain) ;
- **INDUS :** proportion d'acres d'activité non commerciale par ville (l'acre est une ancienne unité de mesure de superficie anglaise) ;
- **CHAS :** variable à 1 si le terrain est délimité par la rivière Charles, 0 sinon ;
- **NOX :** concentration de monoxyde d'azote ;
- **RM :** nombre moyen de pièces par logement ;
- **AGE :** proportion de logements occupés par leur propriétaire construits avant 1940 ;
- **DIS :**  les distances pondérées jusqu'à cinq centres d'emploi de Boston ;
- **RAD :** indice d'accessibilité aux autoroutes radiales ;
- **TAX :** taux d'imposition foncière ;
- **PTRATIO :** ratio élèves/maître par ville ;
- **B :** $1000\cdot (Bk - 0.63)^2$ où $Bk$ est une densité d'habitants par ville ;
- **LSTAT :** statut inférieur de la population.

Concernant la valeur qu'on essaie de trouver à partir des attributs précédent est la valeur médiane des maisons occupées par leur propriétaire en milliers de dollars **MEDV**).
On recherche donc les corrélations des attributs précédents sur le prix médian des maisons.

In [ ]:
# liste de tous les attributs du corpus
attributes = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
              'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

# affichage des données
df = pd.DataFrame(np.concatenate((x_train, y_train[:,None]), axis=1), columns=attributes)
df.head()

À partir des 13 premiers attributs du tableau précédent, on cherche à prédire la dernière colonne.

Les valeurs de la dernière colonne du tableau sont les prix des maisons en milliers de dollars (Vous pouvez remarquer les prix des maisons dans les années 70 !) 

#### Prétraitements des données

On va maintenant prétraiter les données comme dans le cas précédent sur le problème de classification.

On va d'abord caster en float puis normaliser les données en utilisant la standardisation.

**Conversion en float**

In [ ]:
# Conversion des données en type float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

**Normalisation des données**

On peut appliquer le même type de normalisation que les images sur nos données d'apprentissage

In [ ]:
# définition du rééchelonnage
def reechelonnage(X_train, X_test):
    X_train /= X_train.max(axis=0)
    X_test /= X_test.max(axis=0)
    return X_train, X_test

In [ ]:
# définition de la standardisation
def standardisation(X_train, X_test):
    mean = np.mean(X_train, axis=0)
    var = np.var(X_train, axis=0)
    return ((X_train - mean) / var), ((X_test - mean) / var)

On applique la standardisation sur toutes nos données

In [ ]:
X_train, X_test = standardisation(x_train, x_test)
Y_train, Y_test = y_train, y_test

On peut de nouveau afficher les données du corpus d'apprentissage pour voir qu'elles sont normalisées 

In [ ]:
df = pd.DataFrame(np.concatenate((X_train, Y_train[:,None]), axis=1), columns=attributes)
df.head()

#### Création et apprentissage d'un modèle MLP

Dans cette section, nous allons définir un nouveau modèle de réseau de neurones pour répondre à la tâche de régression.

**Définition du modèle**

Dans la cellule suivante, on va définir notre premier modèle de régression linéaire qui va être entraîner sur notre corpus *Boston Housing Price* afin de prédire le prix médian des maisons suivant 13 attributs.

La régression linéaire est représenté par des entrées directement connectées au neurone de sortie. L'idée est d'établir une relation linéaire entre une variable (celle en sortie) par rapport à d'autres variables qui sont en entrée de notre réseau.

In [ ]:
model = Sequential()

# régression linéaire
model.add(Dense(1, input_dim=13, activation='relu'))

# Compiler le modèle en choissisant la bonne fonction de coût et un optimizer
model.compile(loss='mse', # Mean Squared error
              optimizer='sgd',
              metrics=['mae'])

In [ ]:
# Affichage des informations sur le réseau défini au-dessus :
model.summary()

In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=40)

# compléter les arguments de *fit* pour faire un apprentissage sur:
#  200 epochs,
#  avec mini-batches de 32 exemples

history = model.fit(X_train, 
                    Y_train,
                    epochs=200, 
                    batch_size=32,
                    verbose=2,
                    callbacks=[early_stop])

In [ ]:
plt.figure(figsize=(8,3))

# affichage de la valeur de la fonction de perte
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')
plt.show()

### Inférence des données de tests

Évaluons notre modèle

In [ ]:
[loss, mae] = model.evaluate(X_test, Y_test, verbose=0)
print("Testing set Mean Abs Error: ${:.2f}".format(mae*1000))

Affichons les valeurs prédites sur les données de test pour les comparer aux valeurs réelles.

In [ ]:
# Récupération des prédictions
predictions = model.predict(X_test)

In [ ]:
plt.figure(figsize=(12,5))

plt.plot(predictions)
plt.plot(y_test)
plt.legend(['predictions', 'labels'])
plt.show()

### Playground

Modifier le réseau pour avoir deux couches cachées à 64 neurones et analyser les résultats

In [ ]:
model = Sequential()

# régression linéaire
# Couche d'entrée (13) + Première couche cachées (64)
model.add(Dense(64, input_dim=13, activation='relu'))
model.add(Dense(64, activation='relu'))

# Compiler le modèle en choissisant la bonne fonction de coût et un optimizer
model.compile(loss='mse', # Mean Squared error
              optimizer='sgd',
              metrics=['mae'])

# fin de votre code
model.summary()

In [ ]:
optimizer = keras.optimizers.RMSprop(0.001)

model.compile(loss='mse', 
              optimizer=optimizer,
              metrics=['mae'])

early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=20)

history = model.fit(X_train, 
                    Y_train,
                    epochs=500, 
                    batch_size=32,
                    verbose=2,
                    validation_split=0.2,
                    callbacks=[early_stop])

In [ ]:
plt.figure(figsize=(8,5))

plt.xlabel('Epoch')
plt.ylabel('Mean Abs Error [1000$]')
plt.plot(history.epoch, np.array(history.history['mae']), # mean_absolute_error
       label='Train Loss')
plt.plot(history.epoch, np.array(history.history['val_mae']), # val_mean_absolute_error
         label = 'Val loss')
plt.legend()
plt.ylim([0,5])
plt.show()

In [ ]:
plt.figure(figsize=(8,3))

# affichage de la valeur de la fonction de perte
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')
plt.show()

In [ ]:
[_, mae] = model.evaluate(X_test, Y_test, verbose=0)
print("Testing set Mean Abs Error: ${:.2f}".format(mae*1000))

In [ ]:
# Faire des prédictions sur X_test :
predictions = model.predict(X_test)

# Affichage :
plt.figure(figsize=(12,5))
plt.plot(predictions)
plt.plot(y_test)
plt.legend(['predictions', 'labels'])
plt.show()